In [1]:
import tensorflow as tf
import os, argparse, pickle
import numpy as np
import efficientnet.tfkeras as efn
from tqdm import tqdm

In [2]:
def get_args():
    parser = argparse.ArgumentParser()

#     parser.add_argument('--dataset', type=str, default='cifar10',
#                         help='dataset used for training (e.g. cifar10, cifar100, svhn, svhn+extra)')

    parser.add_argument('--epochs', type=int, default=1024, help='number of epochs, (default: 1024)')
    parser.add_argument('--batch_size',  type=int, default=128, help='examples per batch (default: 64)')
    parser.add_argument('--learning_rate', type=float, default=0.256, help='learning_rate, (default: 0.256)')
    parser.add_argument('--lr_decay', type=float, default=0.94, help='learning rate decay, (default: 0.94)')
    parser.add_argument('--tensorboard', action='store_true', help='enable tensorboard visualization')
    parser.add_argument('--resume', action='store_true', help='whether to restore from previous training runs')
    parser.add_argument('--gpus', type=str, default='-1', help='set gpu numbers')

    return parser.parse_known_args(args='')

def scheduler(epoch, lr):
    if not(epoch != 0 and epoch % 2 == 0):
        return lr
    else:
        return lr * 0.94


In [28]:
class get_generator():
    def __init__(self, x, y, shape):
        self.x = x
        self.y = y
        self.shape = shape
        
    def __call__(self):
        for _x, _y in zip(self.x, self.y):
            x = tf.image.resize(_x, self.shape[:2])
            if _y == -1:
                y = _y + 11
            else:
                y = _y / 20
            yield (x, y)

In [29]:
def main():
    config = get_args()[0]
    os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    os.environ['CUDA_VISIBLE_DEVICES'] = config.gpus
    abspath = '/root'
    datapath = os.path.join(abspath, 'datasets/ai_challenge/interspeech20/acoustic')
    trainx = pickle.load(open(os.path.join(datapath, 'trainset_x.pickle'), 'rb'))
    trainy = pickle.load(open(os.path.join(datapath, 'trainset_y.pickle'), 'rb'))
    testx = pickle.load(open(os.path.join(datapath, 'testset_x.pickle'), 'rb'))
    testy = pickle.load(open(os.path.join(datapath, 'trainset_y.pickle'), 'rb'))
    
    shape = 0
    for data in trainx:
        shape += data.shape[0]
    for data in testx:
        shape += data.shape[0]
    shape = (shape // (len(trainx) + len(testx)), trainx[0].shape[1], trainx[0].shape[2])
            
    name = f'b0_RMS_{config.learning_rate}'
    
    tensorboard_path = './tensorboard_log/'+name
    model_save_path = './model_save/'+name
    if not os.path.exists(tensorboard_path):
        os.makedirs(tensorboard_path)
    if not os.path.exists(model_save_path):
        os.makedirs(model_save_path)
    
    
    train_generator = get_generator(trainx, trainy, shape)
    test_generator = get_generator(testx, testy, shape)
    train_dataset = tf.data.Dataset.from_generator(train_generator, (tf.float32, tf.int32), (tf.TensorShape(list(shape)), tf.TensorShape([]))).shuffle(len(trainx)).padded_batch(config.batch_size).prefetch(AUTOTUNE)
    validation_dataset = tf.data.Dataset.from_generator(test_generator, (tf.float32, tf.int32), (tf.TensorShape(list(shape)), tf.TensorShape([]))).shuffle(len(testx)).batch(config.batch_size)

    optimizer = tf.keras.optimizers.RMSprop(lr=config.learning_rate, momentum=0.9)
    model = get_model(shape)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tensorboard_path, histogram_freq=1)
    lrscheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor='val_acc', patience=10, verbose=0, mode='auto', restore_best_weights=True
    )
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        model_save_path, verbose=0,
        save_freq='epoch',
        monitor='val_acc'
    )
    model.summary()
    
    model.compile(optimizer=optimizer, loss=tf.keras.losses.sparse_categorical_crossentropy ,metrics=['acc'])
    model.fit(train_dataset,epochs=config.epochs, validation_data=validation_dataset, callbacks=[
        tensorboard_callback,
        earlystop,
        cp_callback
    ])

In [30]:
def get_model(input_shape):
    inputs = tf.keras.layers.Input(input_shape)
    efficientnet = efn.EfficientNetB0(weights=None,input_shape=input_shape,include_top=False)(inputs)
    x = tf.keras.layers.Conv2D(1280,1)(efficientnet)
    x = tf.keras.layers.MaxPool2D()(x)
    x = tf.keras.layers.Flatten()(x)
    out = tf.keras.layers.Dense(11,activation='softmax')(x)
    return tf.keras.Model(inputs=inputs, outputs=out)

In [31]:
if __name__ == '__main__':
#     model = get_model((None,120,4))
#     model.summary()
    main()

Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 220, 120, 4)]     0         
_________________________________________________________________
efficientnet-b0 (Functional) (None, 7, 4, 1280)        4049852   
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 7, 4, 1280)        1639680   
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 3, 2, 1280)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 7680)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 11)                84491     
Total params: 5,774,023
Trainable params: 5,732,007
Non-trainable params: 42,016
______________________________________

KeyboardInterrupt: 